In [30]:
from selenium import webdriver
from time import sleep
from random import random
import json
from datetime import datetime

load_wait = 3
parse_wait = 4
max_page = 5
max_count_of_products = 100

url = 'https://www.ozon.ru/category/smartfony-15502/'
sort_type = 'sorting=rating'
start_url = f'{url}?{sort_type}'

product_class = 'kr9' # может меняться, возможно xpath будет работать стабильнее
# product_xpath = ''

pth_to_webdriver = 'pth_to_webdriver'

ua_pc = 'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'

def start_driver():
    opts = webdriver.ChromeOptions()
    opts.add_argument('--disable-blink-features=AutomationControlled')
    opts.add_argument('--headless')
    opts.add_argument(ua_pc)
    return webdriver.Chrome(executable_path=pth_to_webdriver, options=opts)

def load_url(driver, url):
    driver.get(url)
    sleep(load_wait*random() + 1)
    driver.execute_script(f'window.scrollTo(5,{int(4000 + 1000*random())});')
    sleep(parse_wait*random() + 1)

def get_products(driver, url):
    load_url(driver, url)
    return driver.find_elements_by_class_name(product_class)

def driver_reset(driver):
    print('Reset driver ...')
    driver.service.stop()
    sleep(load_wait*random() + 1)
    driver = start_driver()
    print('Reset driver successful!!!')
    return driver

def get_top_rate_product_links(driver, start_url, top_rate_product_links):
    print(start_url)
    products = get_products(driver, start_url)
    page_number = 2
    while True:
        for product in products:
            # продукты у которых есть рейтинг имеют больше 2-х тегов svg
            if len(product.find_elements_by_tag_name('svg')) > 2:
                product_link = product.find_element_by_class_name('pk1').get_attribute('href').split('?')[0]
                top_rate_product_links.add(product_link)
            if len(top_rate_product_links) == max_count_of_products:
                print('Закончили собирать продукты.')
                return
        
        # next_url = driver.find_element_by_xpath('//*[@id="layoutPage"]/div/div[2]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div[2]/a').get_attribute('href')
        next_url = f'{url}?page={page_number}&{sort_type}'
        print(next_url)
        
        page_number += 1
        if page_number > max_page:
            return

        driver = driver_reset(driver)
        products = get_products(driver, next_url)
        # products = get_products(driver, next_url)
        
        if len(products) == 0:
            print('Нас заметили!!!')
            print('Бежим!!!')
            return

def get_version(driver, url):
    driver = driver_reset(driver)
    print(url)
    load_url(driver, url)
    e = driver.find_element_by_tag_name('pre')

    k0 = 'widgetStates'
    k1 = 'webCharacteristics-939965-pdpPage2column-2' # ключ верен только для пк версии сайта
    k2 = 'characteristics'

    for x in json.loads(json.loads(e.text)[k0][k1])[k2]:
        for y in x['short']:
            if ('Android' in y['values'][0]['text'] or 'iOS' in y['values'][0]['text']) and y['key'] != 'OSWithoutVer':
                return y['values'][0]['text']
    
    return 'None'

def count_diffrent_version_os(driver, urls, results):
    for url in urls:
        product_url = f'https://www.ozon.ru/api/composer-api.bx/page/json/v2?url={url[19:]}?layout_container=pdpPage2column&layout_page_index=2'
        res = get_version(driver, product_url)
        results.append({'url': url, 'os_version': res})

In [31]:
driver = start_driver()
top_rate_product_links = set()
results = []

In [32]:
get_top_rate_product_links(driver, start_url, top_rate_product_links)

https://www.ozon.ru/category/smartfony-15502/?sorting=rating
https://www.ozon.ru/category/smartfony-15502/?page=2&sorting=rating
Reset driver ...
Reset driver successful!!!
https://www.ozon.ru/category/smartfony-15502/?page=3&sorting=rating
Reset driver ...
Reset driver successful!!!
https://www.ozon.ru/category/smartfony-15502/?page=4&sorting=rating
Reset driver ...
Reset driver successful!!!
Закончили собирать продукты.


In [34]:
count_diffrent_version_os(driver, top_rate_product_links, results)

Reset driver ...
Reset driver successful!!!
https://www.ozon.ru/api/composer-api.bx/page/json/v2?url=/product/smartfon-xiaomi-redmi-10-2022-4-64-gb-belyy-604240842/?layout_container=pdpPage2column&layout_page_index=2
Reset driver ...
Reset driver successful!!!
https://www.ozon.ru/api/composer-api.bx/page/json/v2?url=/product/smartfon-realme-c30-2-32-gb-zelenyy-652816380/?layout_container=pdpPage2column&layout_page_index=2
Reset driver ...
Reset driver successful!!!
https://www.ozon.ru/api/composer-api.bx/page/json/v2?url=/product/smartfon-realme-c25s-4-128-gb-chernyy-322144074/?layout_container=pdpPage2column&layout_page_index=2
Reset driver ...
Reset driver successful!!!
https://www.ozon.ru/api/composer-api.bx/page/json/v2?url=/product/smartfon-apple-iphone-11-64-gb-201995194/?layout_container=pdpPage2column&layout_page_index=2
Reset driver ...
Reset driver successful!!!
https://www.ozon.ru/api/composer-api.bx/page/json/v2?url=/product/smartfon-xiaomi-redmi-10-2022-4-128-gb-belyy-6042

In [36]:
driver.quit()

In [ ]:
current_date = datetime.today().strftime('%y_%m_%d')

with open(f'top_phones_{current_date}.txt', 'w') as f:
    for i, link in enumerate(top_rate_product_links):
        print(f'{i + 1}: {link}', file=f)

with open(f'parse_results_{current_date}.json', 'w') as f:
    json.dump(results, f, indent=4)

In [45]:
import pandas as pd

df = pd.read_json('parse_results_22_12_04.json')
df.os_version.value_counts()

# Android 11.x            46
# Android 12.x            21
# Android 10.x            11
# Android 11 (EMUI 12)    10
# None                     6
# iOS 14                   5
# iOS 13                   1
# Name: os_version, dtype: int64

Android 11.x            46
Android 12.x            21
Android 10.x            11
Android 11 (EMUI 12)    10
None                     6
iOS 14                   5
iOS 13                   1
Name: os_version, dtype: int64